In [31]:
import torch
CUDA = torch.cuda.is_available()
print(CUDA)

False


In [33]:
import time
from lifelines.utils import concordance_index 
import sys
from torch import nn
import survival_analysis_chirag
import numpy as np
import pandas as pd
import network
from torch.utils.data import TensorDataset, Dataset
import torch.utils.data.dataloader as dataloader
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


ds = pd.read_csv('./datasets/whas1638.csv',sep=',')
train = ds[:400]
validation = ds[400:]

# x = train[['age', 'gender', 'bmi', 'chf', 'miord']]
x = train[['1', '2', '3', '4', '5']]
e = train['fstat']
t = train['lenfol']

x = torch.from_numpy(x.as_matrix()).float()
e = torch.from_numpy(e.as_matrix()).float()
t = torch.from_numpy(t.as_matrix())

In [34]:
if CUDA:
    x = x.cuda()
    e = e.cuda()
    t = t.cuda()

In [35]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)
#         m.weight.data.fill_(0)
#         m.bias.data.fill_(1)

def init_weights_for_cox(m):
    if type(m) == nn.Linear:
        m.weight.data.fill_(0)
        m.bias.data.fill_(0)

In [36]:
def train(model, x, e, t, CUDA, optimizer, n_epochs):

    # Initialize Metrics
    c_index = []
    valid_c_index = []

    risk_set = []
    for i in range(len(t)):
        temp = []
        for j in range(len(t)):
            if t[j] >= t[i]:
                temp.append(j)
        risk_set.append(temp)
        
    start = time.time()
    for epoch in range(n_epochs):

        optimizer.zero_grad()
        # print("x: ", x)
        outputs = model(x)

        loss = negative_log_likelihood(outputs, e, risk_set, CUDA)
        loss.backward()
        optimizer.step()


        print(loss.cpu().data.numpy())
        
        ci_train = get_concordance_index(outputs, t, e)
        c_index.append(ci_train)
        torch.cuda.empty_cache()
                   
        print('Finished Training with %d iterations in %0.2fs' % (epoch + 1, time.time() - start))
    
    metrics = {}
    metrics['c-index'] = c_index
    return metrics


In [37]:
def negative_log_likelihood(risk, E, risk_set, CUDA):
    
    import numpy as np
    
#     new_risk = []
#     for i in range(len(risk_set)):
#         new_risk.append(risk[risk_set[i]])
        
#     log_risk = []
#     for i in range(len(new_risk)):
#         temp = torch.logsumexp(new_risk[i], 0)
#         log_risk.append(temp)

    lnumerator = risk
    
    idxs = range(risk.shape[0])
    
    
    ldenominator = []
    
    for i in range(len(idxs)):
        
        ldenominator.append(torch.logsumexp(risk[risk_set[i]], dim=0))
    
    ldenominator = torch.cat(ldenominator).reshape(-1, 1)
    
    
    likelihoods = lnumerator - ldenominator
    
    E =  np.where(E.cpu().data.numpy()==1)[0]
    

    
    neg_likelihood = - torch.sum(likelihoods[E])
    


    return neg_likelihood


In [38]:
def get_concordance_index(x, t, e, **kwargs):
    x = x.detach().cpu().numpy()
    t = t.detach().cpu().numpy()
    e = e.detach().cpu().numpy()
    computed_hazard = np.exp(x)

    return concordance_index(t,-1*computed_hazard,e)


In [40]:

# For CPH, set cox argument as True
print("CPH model")
n_in = x.shape[1]

layers_sizes = [n_in, 1]

# Construct Neural Network
layers = []
for i in range(len(layers_sizes)-2):
    layers.append(nn.Linear(layers_sizes[i],layers_sizes[i+1]))
    layers.append(nn.ReLU())

layers.append(nn.Linear(layers_sizes[-2], layers_sizes[-1]))
my_network = nn.Sequential(*layers)
#my_network.apply(init_weights)

#optimizer = optimizer = torch.optim.SGD(my_network.parameters(), lr=learning_rate, momentum=momentum, weight_decay=L2_reg, nesterov=True)

optimizer = torch.optim.Adam(my_network.parameters(), lr=1e-3)
my_network.train()
if CUDA:
    my_network.cuda()

# If you have validation data, you can add it as the valid_dataloader parameter to the function
n_epochs = 10
metrics = train(my_network, x, e, t, CUDA, optimizer, n_epochs)
print()

print("Done")

CPH model
3118.769
Finished Training with 1 iterations in 0.03s
3111.0977
Finished Training with 2 iterations in 0.05s
3103.429
Finished Training with 3 iterations in 0.08s
3095.763
Finished Training with 4 iterations in 0.11s
3088.1013
Finished Training with 5 iterations in 0.14s
3080.4421
Finished Training with 6 iterations in 0.17s
3072.7874
Finished Training with 7 iterations in 0.20s
3065.1362
Finished Training with 8 iterations in 0.24s
3057.4888
Finished Training with 9 iterations in 0.27s
3049.8452
Finished Training with 10 iterations in 0.30s

Done


In [41]:
my_network

Sequential(
  (0): Linear(in_features=5, out_features=1, bias=True)
)

In [42]:
print("Train C-Index:", metrics['c-index'])

Train C-Index: [0.2861588418346638, 0.2860911919902584, 0.28611374193839356, 0.2866098407973662, 0.2866098407973662, 0.2866098407973662, 0.2865647409010959, 0.2866098407973662, 0.2868578902268525, 0.2868127903305822]
